In [1]:
#MASK rcnn-objcect detection
#from google.colab import drive
#drive.mount('/content/drive')

In [4]:
#TrainDir = "/content/drive/MyDrive/Content/DriveAIML/capstone/jpg_reshaped_train/"
#TestDir='/content/drive/MyDrive/Content/DriveAIML/capstone/jpg_reshaped_test/'
#sublabel='/content/drive/MyDrive/Content/DriveAIML/capstone/stage_2_sample_submission.csv'
#basepath = "/content/drive/MyDrive/Content/DriveAIML/capstone/"

#local path
TrainDir = 'C:/Users/vao5kor/Desktop/Python/olympus/Capstone project/rsna-pneumonia-detection-challenge/jpg_reshaped_train/'
TestDir='C:/Users/vao5kor/Desktop/Python/olympus/Capstone project/rsna-pneumonia-detection-challenge/jpg_reshaped_test/'
sublabel='C:/Users/vao5kor/Desktop/Python/olympus/Capstone project/rsna-pneumonia-detection-challenge/stage_2_sample_submission.csv'
basepath = "C:/Users/vao5kor/Desktop/Python/olympus/Capstone project/rsna-pneumonia-detection-challenge/"
modeldir = 'C:/Users/vao5kor/Downloads/Mask_RCNN/'

In [3]:
import os, cv2,copy, gc
import pickle
import glob2 as glob
import pandas as pd
import numpy as np
import datetime
from tqdm import tqdm_notebook
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.python.client import device_lib
from tensorflow import keras
from tensorflow.keras.applications import VGG16
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import models, layers, Model, losses
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.layers import Input, Concatenate, Dense, Dropout, Flatten, Activation, BatchNormalization
 

ModuleNotFoundError: No module named 'glob2'

In [ ]:
#tf.compat.v1.variable_scope()
#tf.compat.v1.variable_scope()
 

In [ ]:
filename =  basepath+'RezisedLabel.pickle'
with open(filename, "rb") as input_file:
  y_train = pickle.load(input_file)

In [ ]:
#trainimg = glob.glob(TrainDir+'*.jpg') #Getting all images in this folder
#testimg=glob.glob(TestDir+"*.jpg") #

In [ ]:
 #len(trainimg)

In [ ]:
y_train.head(5)


In [ ]:
y_train.size

In [ ]:
class_info_df = pd.read_csv(basepath+'stage_2_detailed_class_info.csv')
sub_info_df=pd.read_csv(sublabel)#test label or submission 

In [ ]:
## Merging the two datasets
#In class detailed info dataset are given the detailed information about 
#the type of positive or negative class associated with a certain patient.
#In y_train labels dataset are given the patient ID and the window (x min, y min, width and height of the) 
#containing evidence of pneumonia
train_class_df = y_train.merge(class_info_df, left_on= ('patientId'), right_on=('patientId'), how='inner').drop_duplicates()
train_class_df.sample(5)




In [ ]:
#reading image 500 samples traindir
sampNum = 500 # Number of samples to import
X = [] # Empty to read training images
x1=[]
print(type(x1))
print("Before sampling\n")
print("Total training unique patient ids: ",train_class_df['patientId'].nunique())
print("Number of images in train set:", len(os.listdir(TrainDir)))
print("\n")
print("Total training labels {}" .format(train_class_df.shape))
#display(distTab(train_class_df, "Target")) # Dsitribution of target before sampling

print("\n\nAfter sampling\n")
Y = train_class_df.sample(n=sampNum, random_state=2) # Sampling
Y.reset_index(drop=True, inplace=True)
print("Total sampled training labels {}" .format(Y.shape))
#display(distTab(Y.sample(n=sampNum), "Target"))  # Dsitribution of target after sampling

for indx in Y.patientId:
    
    try:
       #print(indx)
       filepath=TrainDir+indx+'.jpg'
       image = cv2.imread(filepath)
       X.append(indx)
       x1.append(image)
      # print(filepath)
    except: 
        continue
zipbObj = zip(X, x1)        
 

X=dict(zipbObj)#takes zip object
print("\n\nTotal sampled training images {}" .format(len(X)))
Y = Y[Y.patientId.isin(X.keys())]
print("Total training labels {}" .format(Y.shape))

In [ ]:
#reading image 100 samples test dir
sampNum = 10 # Number of samples to import
Xtest = [] # Empty to read training images
x1test=[] 
#print("Total training unique patient ids: ",train_class_df['patientId'].nunique())
print("Total training unique patient ids: ",sub_info_df['patientId'].nunique())
print("Number of images in test set:", len(os.listdir(TestDir)))
 
print("\n")
print("Total training labels {}" .format(sub_info_df.shape))
#print("Total training labels {}" .format(train_class_df.shape))
#Ytest = train_class_df.sample(n=sampNum, random_state=2) # Sampling
Ytest = sub_info_df.sample(n=sampNum, random_state=2) # Sampling
Ytest.reset_index(drop=True, inplace=True)
print("Total sampled testing labels {}" .format(Ytest.shape))
#display(distTab(Y.sample(n=sampNum), "Target"))  # Dsitribution of target after sampling

for indx1 in Ytest.patientId:
    
    try:
      # print(indx1)
       filepath1=TestDir+indx1+'.jpg'
       image1 = cv2.imread(filepath1)
       Xtest.append(indx1)
       x1test.append(image1)
      # print(filepath1)
    except: 
        continue
zipbObj1 = zip(Xtest, x1test)        
 

Xtest=dict(zipbObj1)#takes list object
#print(Xtest)
print("\n\nTotal sampled test images {}" .format(len(Xtest)))
Ytest = Ytest[Ytest.patientId.isin(Xtest.keys())]
print("Total test labels {}" .format(Ytest.shape))

In [ ]:
# Shape of images
print(len(X))
print(len(Xtest))
print(len(Y))
print(len(Ytest))


In [ ]:
Xdf = pd.DataFrame({'patientId':X.keys(),
                   'imageArray':X.values()})
dattrain = Y.merge(Xdf, on='patientId', how='left')

#test
Xtestdf = pd.DataFrame({'patientId':Xtest.keys(),
                   'imageArray':Xtest.values()})
dattest = Ytest.merge(Xtestdf, on='patientId', how='left')

In [ ]:
dattrain.head(10)

In [ ]:
 dattest.head(10)

In [ ]:
#
#test_class_df = y_train.merge(sub_info_df, left_on= ('patientId'), right_on=('patientId'), how='inner').drop_duplicates()
#train_class_df.describe

MSK RCC model


In [ ]:
#download from  git
!git clone https://www.github.com/matterport/Mask_RCNN.git
os.chdir('Mask_RCNN')
!python setup.py -q install





In [ ]:
# Import Mask RCNN in root folder

import sys,os
# Root directory of the project
ROOT_DIR = os.path.abspath("../")
print(ROOT_DIR)
sys.path.append(ROOT_DIR)
sys.path.append(os.path.join(basepath, 'Mask_RCNN'))  # To find local version of the library
from mrcnn.config import Config
from mrcnn import utils
from tensorflow.keras.layers import InputLayer, Input
import mrcnn.utils as utils
import mrcnn.model as modellib
from mrcnn import visualize
from mrcnn.model import log
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM

In [ ]:
#pip install tensorflow --upgrade --force-reinstall

In [ ]:
#download weights
### Download COCO pre-trained weights
#!wget --quiet https://github.com/matterport/Mask_RCNN/releases/download/v2.0/mask_rcnn_coco.h5
#!ls -lh mask_rcnn_coco.h5

COCO_WEIGHTS_PATH = basepath+"mask_rcnn_coco.h5"

In [ ]:
#img and annotation looping
def get_img_fps(img_dir):
    img_fps = glob.glob(img_dir+'*.jpg')
    return list(set(img_fps))

def parse_dataset(img_dir, anns): 
    image_fps = get_img_fps(img_dir)
    print(image_fps)
    image_annotations = {fp: [] for fp in image_fps}
    for index, row in anns.iterrows(): 
        fp = os.path.join(img_dir, row['patientId']+'.jpg')  #need to check this code
        image_annotations[fp].append(row)
    return image_fps, image_annotations 

In [ ]:
class DetectorConfig(Config):
    """Configuration for training pneumonia detection on the RSNA pneumonia dataset.
    Overrides values in the base Config class.
    """
    
    # Give the configuration a recognizable name  
    NAME = 'pneumonia'
    
    # Train on 1 GPU and 8 images per GPU. We can put multiple images on each
    # GPU because the images are small. Batch size is 8 (GPUs * images/GPU).
    GPU_COUNT = 1
    IMAGES_PER_GPU = 8
    
    BACKBONE = 'resnet50'
    
    NUM_CLASSES = 2  # background + 1 pneumonia classes
    
    IMAGE_MIN_DIM = 256
    IMAGE_MAX_DIM = 256
    RPN_ANCHOR_SCALES = (32, 64, 128, 256)
    TRAIN_ROIS_PER_IMAGE = 32
    MAX_GT_INSTANCES = 3
    DETECTION_MAX_INSTANCES = 3
    DETECTION_MIN_CONFIDENCE = 0.7
    DETECTION_NMS_THRESHOLD = 0.1

    STEPS_PER_EPOCH = 100
    
config = DetectorConfig()
config.display()

In [ ]:
class DetectorDataset(utils.Dataset):
    """Dataset class for training pneumonia detection on the RSNA pneumonia dataset.
    """

    def __init__(self, image_fps, image_annotations, orig_height, orig_width):
        super().__init__(self)
        
        # Add classes
        self.add_class('pneumonia', 1, 'Lung Opacity')
        
        # add images 
        for i, fp in enumerate(image_fps):
            annotations = image_annotations[fp]
            self.add_image('pneumonia', image_id=i, path=fp, 
                           annotations=annotations, orig_height=orig_height, orig_width=orig_width)
            
    def image_reference(self, image_id):
        info = self.image_info[image_id]
        return info['path']

    def load_image(self, image_id):
        info = self.image_info[image_id]
        fp = info['path']
        ds = pydicom.read_file(fp)
        image = ds.pixel_array
        # If grayscale. Convert to RGB for consistency.
        if len(image.shape) != 3 or image.shape[2] != 3:
            image = np.stack((image,) * 3, -1)
        return image

    def load_mask(self, image_id):
        info = self.image_info[image_id]
        annotations = info['annotations']
        count = len(annotations)
        if count == 0:
            mask = np.zeros((info['orig_height'], info['orig_width'], 1), dtype=np.uint8)
            class_ids = np.zeros((1,), dtype=np.int32)
        else:
            mask = np.zeros((info['orig_height'], info['orig_width'], count), dtype=np.uint8)
            class_ids = np.zeros((count,), dtype=np.int32)
            for i, a in enumerate(annotations):
                if a['Target'] == 1:
                    x = int(a['x'])
                    y = int(a['y'])
                    w = int(a['width'])
                    h = int(a['height'])
                    mask_instance = mask[:, :, i].copy()
                    cv2.rectangle(mask_instance, (x, y), (x+w, y+h), 255, -1)
                    mask[:, :, i] = mask_instance
                    class_ids[i] = 1
        return mask.astype(np.bool), class_ids.astype(np.int32)

In [ ]:
anns=y_train

In [ ]:
anns.head(5)

In [ ]:
image_fps, image_annotations = parse_dataset(TrainDir, anns=anns)

In [ ]:
hyper_paramters_comb={
    'backbone':['resnet50'],
    'learning_rate':[0.005],
    'batch_size':[8],
    'epochs':[10],
    'det_min_conf':[0.9],
    'det_nms_th':[0.8],
    'rpn_nms_th':[0.7],
    'steps_per_epoch':[135],
    'layers': ['heads']
}

hpc=pd.DataFrame(hyper_paramters_comb)

hpc['learning_rate'] = hpc['learning_rate'].astype(np.float32)
hpc['det_min_conf'] = hpc['det_min_conf'].astype(np.float32)
hpc['det_nms_th'] = hpc['det_nms_th'].astype(np.float32)
hpc['rpn_nms_th'] = hpc['rpn_nms_th'].astype(np.float32)

hpc.head()

In [ ]:
NUM_EPOCHS = 16
LEARNING_RATE = 0.006

# Train Mask-RCNN Model 
import warnings 
warnings.filterwarnings("ignore")

model = modellib.MaskRCNN(mode='training', config=config, model_dir=modeldir)

# Exclude the last layers because they require a matching number of classes
model.load_weights(COCO_WEIGHTS_PATH, by_name=True, exclude=[
    "mrcnn_class_logits", "mrcnn_bbox_fc", "mrcnn_bbox", "mrcnn_mask"])


In [ ]:
%%time
## first epochs with higher lr to speedup the learning
model.train(dattrain, dattest,
            learning_rate=LEARNING_RATE*2,
            epochs=2,
            layers='all',
            augmentation=None)  ## no need to augment yet

In [ ]:
import os
# select trained model 
dir_names = next(os.walk(model.model_dir))[1]
key = config.NAME.lower()
dir_names = filter(lambda f: f.startswith(key), dir_names)
dir_names = sorted(dir_names)

if not dir_names:
    import errno
    raise FileNotFoundError(
        errno.ENOENT,
        "Could not find model directory under {}".format(self.model_dir))
    
fps = []
# Pick last directory
for d in dir_names: 
    dir_name = os.path.join(model.model_dir, d)
    # Find the last checkpoint
    checkpoints = next(os.walk(dir_name))[2]
    checkpoints = filter(lambda f: f.startswith("mask_rcnn"), checkpoints)
    checkpoints = sorted(checkpoints)
    if not checkpoints:
        print('No weight files in {}'.format(dir_name))
    else:
        checkpoint = os.path.join(dir_name, checkpoints[-1])
        fps.append(checkpoint)

model_path = sorted(fps)[-1]
print('Found model {}'.format(model_path))